In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
session = get_active_session()
role = session.get_current_role().strip('"')
db = session.get_current_database()
schema = session.get_current_schema()
print(session)


In [ ]:
USE ROLE {{role}};
USE DATABASE {{db}};
USE SCHEMA {{schema}};

In [ ]:
CREATE OR REPLACE TABLE clinical_trial_raw (
    study_id STRING, 
    study_data VARIANT
    );

In [ ]:
CREATE OR REPLACE NETWORK RULE CLINICAL_TRIALS_GOV_PUBLIC_DATA_RULE 
MODE = EGRESS 
TYPE = HOST_PORT 
VALUE_LIST = ('clinicaltrials.gov')

In [ ]:
CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION CLINICAL_TRIALS_GOV_PUBLIC_DATA_INTEGRATION
ALLOWED_NETWORK_RULES = (CLINICAL_TRIALS_GOV_PUBLIC_DATA_RULE)
ENABLED = TRUE;

In [ ]:
CREATE OR REPLACE FUNCTION FETCH_ENDPOINT (URL STRING)
returns string
language python
runtime_version=3.9
handler = 'fetch_data'
external_access_integrations = (CLINICAL_TRIALS_GOV_PUBLIC_DATA_INTEGRATION)
packages = ('requests')
as
$$
import requests
import _snowflake
session = requests.Session()
def fetch_data(URL):
    response = requests.get(URL)
    return response.json()
$$;

In [ ]:
SET url  = 'https://clinicaltrials.gov/api/v2/studies/NCT02953860'

In [ ]:
SELECT FETCH_ENDPOINT($url);

In [ ]:
INSERT INTO clinical_trial_raw 
SELECT split_part($url, '/', -1)::STRING as study_id, 
fetch_endpoint($url)::VARIANT as study_data;


In [ ]:
select * from clinical_trial_raw;